In [2]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
llm_model = "gpt-3.5-turbo-0301"

In [5]:
import shutil

try:
    os.mkdir("chroma")
except:
    shutil.rmtree("chroma")

In [9]:
from urllib.request import urlretrieve
url = ("https://primarywater.blob.core.windows.net/tunga/MasakhaNEWS_News_Topic_Classification_for_African_.pdf")
filename = "document.pdf"

urlretrieve(url, filename)

('document.pdf', <http.client.HTTPMessage at 0x7f74fbcfa2e0>)

In [10]:
file = "document.pdf"

In [24]:
from langchain.document_loaders import PyPDFLoader
file = "document.pdf"
docs = PyPDFLoader(file)

docs = docs.load()

In [25]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [26]:
splits = text_splitter.split_documents(docs)

In [27]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [37]:
persist_directory = 'docs/chroma/'

!rm -rf ./chroma  # remove old database files if any

In [39]:
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)
vectordb.persist()

In [35]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory = 'chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

In [54]:
print(vectordb._collection.count())

265


In [45]:
from langchain.chat_models import ChatOpenAI

llm_name = "gpt-3.5-turbo-0301"

llm = ChatOpenAI(model_name=llm_name, temperature=0)

In [46]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [57]:
question = "What is Mashkahne in summary"

result = qa_chain({"query": question})

In [58]:
result["result"]

'MasakhaNEWS is a new benchmark dataset for news topic classification covering 16 languages widely spoken in Africa. The paper provides an evaluation of baseline models by training classical machine learning models and fine-tuning several language models. The authors explore several alternatives to full fine-tuning of language models that are better suited for zero-shot and few-shot learning. The evaluation in zero-shot setting shows the potential of prompting ChatGPT for news topic classification in low-resource African languages, achieving an average performance of 70 F1 points without leveraging additional supervision like MAD-X.'